In [1]:
#!rm -r /home/jovyan/.local/share/jupyter/kernels/nilmtk
!mamba install tensorflow-gpu==2.10 -y -q
# !mamba install tensorflow-gpu==2.11.0 -y -q
!pip uninstall -y -q nilmtk nilm_metadata
#!pip install -q pandas networkx tables scikit-learn hmmlearn pyyaml matplotlib==3.1.3 xgboost pyts
# Trick to install NILM regardless of its dependencies
#!pip install --no-cache -U git+https://github.com/nilmtk/nilm_metadata.git
#!pip install --no-cache -U git+https://github.com/nilmtk/nilmtk.git@0.4.3
!python3 -m pip install --no-deps git+https://github.com/nilmtk/nilmtk@master
!python3 -m pip install --no-deps git+https://github.com/nilmtk/nilm_metadata@master

Preparing transaction: ...working... done
Verifying transaction: ...working... done
Executing transaction: ...working... By downloading and using the CUDA Toolkit conda packages, you accept the terms and conditions of the CUDA End User License Agreement (EULA): https://docs.nvidia.com/cuda/eula/index.html

By downloading and using the cuDNN conda packages, you accept the terms and conditions of the NVIDIA cuDNN EULA -
  https://docs.nvidia.com/deeplearning/cudnn/sla/index.html

done
  Cloning https://github.com/nilmtk/nilmtk (to revision master) to /tmp/pip-req-build-fxirbq2g
  Running command git clone --filter=blob:none --quiet https://github.com/nilmtk/nilmtk /tmp/pip-req-build-fxirbq2g
  Resolved https://github.com/nilmtk/nilmtk to commit b2c514479cef478cab872cb635056da08d5352a1
  Preparing metadata (setup.py) ... done
  Created wheel for nilmtk: filename=nilmtk-0.4.0.dev1+git.b2c5144-py3-none-any.whl size=279177 sha256=4f257c9ba5d11734665c3c4d762c64070b7db516db1089afd9f0cb52d6e7c9

In [3]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# from nilmtk import DataSet
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from collections import defaultdict
from joblib import Memory
import pickle
from pprint import pprint

#from nilmtk import DataSet
import multiprocessing as mp

from typing import Dict

!pip install -q scikit-learn-intelex
#from sklearnex import patch_sklearn
#patch_sklearn()

from sklearn import pipeline, metrics, linear_model, model_selection, multioutput, tree, ensemble, neural_network
!pip install xgboost -q
import xgboost as xgb

import numpy as np
import pickle
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.metrics import accuracy_score
from sklearn import datasets, linear_model
from sklearn.model_selection import cross_val_predict, train_test_split
from sklearn import metrics
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import LearningRateScheduler

import numpy as np
#import graphviz
import keras
from sklearn.utils import class_weight
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Activation, Conv2D, MaxPooling2D, BatchNormalization
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import confusion_matrix
from keras.preprocessing.image import ImageDataGenerator
#from ann_visualizer.visualize import ann_viz
from matplotlib import pyplot as plt 
from sklearn.metrics import multilabel_confusion_matrix, classification_report, confusion_matrix
# GPU goes brrrrrrrrrrrrrrrrrrrr
from tensorflow.keras import mixed_precision
mixed_precision.set_global_policy('mixed_float16')

!pip3 install tqdm -q
from tqdm.notebook import tqdm
import NUK

# import garbage collector
import gc
import time


physical_devices = tf.config.list_physical_devices('GPU')

if physical_devices:
    print(physical_devices)
    tf.config.experimental.set_memory_growth(physical_devices[0], True)
else:
    print("No GPU found, model will train on CPU.")


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


# Preprocessing

# Models

In [10]:
# Parameters

epochs =300
window_size = 2700
batch_size = 128
NmDevices = 78
# table_of_options = [1,2,3,4]
k = 0.25
# Define a learning rate scheduler function
def scheduler(epoch, lr):
    return lr
    if epoch == 0 or epoch == 1:
        return lr
    if epoch == 35:
        return lr *0.5
    else:
        return lr

RANDOM_SEED = 42
test_size = 0.2
lambda_l2=0
function = "GRU"

In [5]:
data= pd.read_pickle("../../Energy_graph/data/training_data/processed/X_Y_wsize2700_upper32_gap3600.pkl")
data_syn_ideal = pd.read_pickle("../../Energy_graph/data/training_data/synthetic/X_Y_wsize2700_numW_50000_upper32_gap3600_ideal.pkl") 
data_syn_unmetered = pd.read_pickle("../../Energy_graph/data/training_data/synthetic/X_Y_wsize2700_numW_50000_upper32_gap3600_unmetered.pkl")

labels = pd.read_pickle("../../Energy_graph/data/labels_new.pkl")


# Separate the tuples into X and y
X_syn_ideal = np.array([i[0] for i in data_syn_ideal])
y_syn_ideal = np.array([i[1] for i in data_syn_ideal])

X_syn_unmetered = np.array([i[0] for i in data_syn_unmetered])
y_syn_unmetered = np.array([i[1] for i in data_syn_unmetered])

X = np.array([i[0] for i in data])
y = np.array([i[1] for i in data])

y = y.astype(int)

X = np.concatenate((X, X_syn_ideal,X_syn_unmetered), axis=0)
y = np.concatenate((y, y_syn_ideal, y_syn_unmetered), axis=0)

# X_test = np.array([i[0] for i in data_test])
# y_test = np.array([i[1] for i in data_test])
def normalize(X):
    max_value = 0

    for x in X:
        v = np.max(x)
        if v > max_value:
            max_value = v
    return X / max_value

    




In [20]:
len(labels)

78

### InceptionTime

### InceptionTime class

In [13]:
class_weighs_pre = NUK.class_weights_tool(y)

class Classifier_INCEPTION:

    def __init__(self, output_directory, input_shape, nb_classes, iteration, verbose=False, build=True, batch_size=64,
                 nb_filters=32, use_residual=True, use_bottleneck=True, depth=6, kernel_size=41, nb_epochs=1500, lr=0.001):

        self.output_directory = output_directory

        self.nb_filters = nb_filters
        self.use_residual = use_residual
        self.use_bottleneck = use_bottleneck
        self.depth = depth
        self.kernel_size = kernel_size - 1
        self.callbacks = None
        self.batch_size = batch_size
        self.bottleneck_size = 32
        self.nb_epochs = nb_epochs
        self.lr = lr
        self.iteration = iteration

        if build == True:
            self.model = self.build_model(input_shape, nb_classes)
            # if (verbose == True):
                # self.model.summary()
            self.verbose = verbose
            self.model.save_weights(self.output_directory + 'model_init.hdf5')

    def _inception_module(self, input_tensor, stride=1, activation='linear'):

        if self.use_bottleneck and int(input_tensor.shape[-1]) > 1:
            input_inception = keras.layers.Conv1D(filters=self.bottleneck_size, kernel_size=1,
                                                  padding='same', activation=activation, use_bias=False)(input_tensor)
        else:
            input_inception = input_tensor

        # kernel_size_s = [3, 5, 8, 11, 17]
        kernel_size_s = [self.kernel_size // (2 ** i) for i in range(3)]

        conv_list = []

        for i in range(len(kernel_size_s)):
            conv_list.append(keras.layers.Conv1D(filters=self.nb_filters, kernel_size=kernel_size_s[i],
                                                 strides=stride, padding='same', activation=activation, use_bias=False)(
                input_inception))

        max_pool_1 = keras.layers.MaxPool1D(pool_size=3, strides=stride, padding='same')(input_tensor)

        conv_6 = keras.layers.Conv1D(filters=self.nb_filters, kernel_size=1,
                                     padding='same', activation=activation, use_bias=False)(max_pool_1)

        conv_list.append(conv_6)

        x = keras.layers.Concatenate(axis=2)(conv_list)
        x = keras.layers.BatchNormalization()(x)
        x = keras.layers.Activation(activation='relu')(x)
        return x

    def _shortcut_layer(self, input_tensor, out_tensor):
        shortcut_y = keras.layers.Conv1D(filters=int(out_tensor.shape[-1]), kernel_size=1,
                                         padding='same', use_bias=False)(input_tensor)
        shortcut_y = keras.layers.BatchNormalization()(shortcut_y)

        x = keras.layers.Add()([shortcut_y, out_tensor])
        x = keras.layers.Activation('relu')(x)
        return x

    def build_model(self, input_shape, nb_classes):
        input_layer = keras.layers.Input(input_shape)

        x = input_layer
        input_res = input_layer

        for d in range(self.depth):

            x = self._inception_module(x)

            if self.use_residual and d % 3 == 2:
                x = self._shortcut_layer(input_res, x)
                input_res = x

        gap_layer = keras.layers.GlobalAveragePooling1D()(x)

        output_layer = keras.layers.Dense(nb_classes, activation='sigmoid')(gap_layer)

        model = keras.models.Model(inputs=input_layer, outputs=output_layer)

        model.compile(loss='binary_crossentropy', optimizer=keras.optimizers.Adam(learning_rate=self.lr),
                      metrics=['accuracy', NUK.F1Score, NUK.WeightedF1Score(class_weighs_pre)])

        reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='loss', factor=0.25, patience=10, min_lr=0.0001)
        early_stopping = keras.callbacks.EarlyStopping(patience=15, restore_best_weights=True, monitor='loss')

        file_path = self.output_directory + f'best_model_{self.iteration}.hdf5'

        model_checkpoint = keras.callbacks.ModelCheckpoint(filepath=file_path, monitor='loss',
                                                           save_best_only=True)
        # lr_scheduler = LearningRateScheduler(scheduler)
        self.callbacks = [reduce_lr, model_checkpoint, early_stopping]

        return model

    def fit(self, x_train, y_train):
       
        # x_val and y_val are only used to monitor the test loss and NOT for training

        if self.batch_size is None:
            mini_batch_size = int(min(x_train.shape[0] / 10, 16))
        else:
            mini_batch_size = self.batch_size

        self.model.fit(x_train, y_train, batch_size=mini_batch_size, epochs=self.nb_epochs,
                                  verbose=self.verbose, callbacks=self.callbacks)


        self.model.save(self.output_directory + f'last_model_{self.iteration}.hdf5')

        keras.backend.clear_session()

        return None

    def predict(self, x_test):
        model_path = self.output_directory + f'last_model_{self.iteration}.hdf5'
        model = keras.models.load_model(model_path, custom_objects={'F1Score': NUK.F1Score, "WeightedF1": NUK.WeightedF1Score(class_weighs_pre)})
        y_pred = model.predict(x_test, batch_size=self.batch_size)
    #     # if return_df_metrics:
    #     #     y_pred = np.argmax(y_pred, axis=1)
    #     #     df_metrics = calculate_metrics(y_true, y_pred, 0.0)
    #     #     return df_metrics
    #     # else:
    #     #     test_duration = time.time() - start_time
    #     #     save_test_duration(self.output_directory + 'test_duration.csv', test_duration)
    #     #     return y_pred
        return y_pred


### Single model

In [ ]:

kf = KFold(n_splits=5, shuffle=True, random_state=RANDOM_SEED)
evaluation_results = []
predictions = []
class_weights_all = NUK.class_weights_tool(y)
itr = 0
for train_index, test_index in tqdm(kf.split(X)):
    # print("Train indices:", train_index, "Test indices:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    X_train = normalize(X_train)
    X_test = normalize(X_test)
    # model = NUK.PC0_reg(NmDevices, window_size, function, 128, k, lambda_l2=lambda_l2)
    print(batch_size, epochs)
    model = Classifier_INCEPTION(output_directory="./models/", input_shape=(window_size, 1), nb_classes=78, verbose=True, build=True, batch_size=batch_size, nb_epochs=epochs, lr=0.001, depth=6)
    model.fit(X_train, y_train)
    # model.model.predict(X_test, batch_size=batch_size)

    # Evaluate the model
    y_pred = model.predict(X_test)
    y_pred_tf = (y_pred > 0.5)

    predictions.append((y_pred_tf, y_test, y_pred))
    t = (y_pred_tf, y_test, y_pred)
    with open(f"./predictions/test_inception_pickle_{itr}.pkl", 'wb') as f:
        pickle.dump(t, f)

    itr += 1
    report = metrics.classification_report(y_test, y_pred_tf, target_names=labels, zero_division=0, output_dict=True)
    print(report["weighted avg"])
    # report_DF = NUK.ClassificationReportToDF(report, labels)  
    # report_DF.rename(columns={report_DF.columns[0]: "device"}, inplace=True)
    evaluation_results.append(report)
    gc.collect()
    tf.keras.backend.clear_session()

for i in range(len(evaluation_results)):
    if i == 0:
        df = pd.DataFrame(evaluation_results[i]).T
    else:
        df = df + pd.DataFrame(evaluation_results[i]).T


df = df / 5
import os
# check if directory exsits
if not os.path.exists(f"../../Energy_graph/data/results/{window_size}"):
    os.makedirs(f"../../Energy_graph/data/results/{window_size}")
if lambda_l2 == 0:
    df.to_csv(f"../../Energy_graph/data/results/{window_size}/Inception_epochs{epochs}_batch_size{batch_size}_k{k}_classes{NmDevices}_{function}_watts_syn.csv")
else:
    df.to_csv(f"../../Energy_graph/data/results/w_size{window_size}_epochs{epochs}_batch_size{batch_size}_k{k}_classes{NmDevices}_{function}_watts_syn_reg{lambda_l2}.csv")



    


### Ensemble

In [14]:
num_splits = 3
kf = KFold(n_splits=num_splits, shuffle=True, random_state=RANDOM_SEED)
evaluation_results = []
predictions = []
class_weights_all = NUK.class_weights_tool(y)
for fold,(train_index, test_index) in tqdm(enumerate(kf.split(X))):
    # print("Train indices:", train_index, "Test indices:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    # normalization
    X_train = normalize(X_train)
    X_test = normalize(X_test)
    model_predictions = []

    print(batch_size, epochs)
    for i in range(5):

        model = Classifier_INCEPTION(output_directory=f"./models/{fold}/", input_shape=(window_size, 1), nb_classes=78, verbose=True, build=True, batch_size=batch_size, nb_epochs=epochs, lr=0.001, depth=6, iteration=i)
        model.fit(X_train, y_train)

        # Evaluate the model
        y_pred = model.predict(X_test)
        model_predictions.append(y_pred)

    y_pred = np.mean(model_predictions, axis=0)
    y_pred_tf = (y_pred > 0.5)

    predictions.append((y_pred_tf, y_test, y_pred))
    t = (y_pred_tf, y_test, y_pred)
    with open(f"./predictions/test_inception_pickle_{fold}.pkl", 'wb') as f:
        pickle.dump(t, f)

    report = metrics.classification_report(y_test, y_pred_tf, target_names=labels, zero_division=0, output_dict=True)
    print(report["weighted avg"])
    
    evaluation_results.append(report)
    gc.collect()
    tf.keras.backend.clear_session()

for i in range(len(evaluation_results)):
    if i == 0:
        df = pd.DataFrame(evaluation_results[i]).T
    else:
        df = df + pd.DataFrame(evaluation_results[i]).T


df = df / num_splits
import os
# check if directory exsits
if not os.path.exists(f"../../Energy_graph/data/results/{window_size}"):
    os.makedirs(f"../../Energy_graph/data/results/{window_size}")
if lambda_l2 == 0:
    df.to_csv(f"../../Energy_graph/data/results/{window_size}/NNE_epochs{epochs}_batch_size{batch_size}_k{k}_classes{NmDevices}_{function}_watts_syn_3fold.csv")
else:
    df.to_csv(f"../../Energy_graph/data/results/w_size{window_size}_epochs{epochs}_batch_size{batch_size}_k{k}_classes{NmDevices}_{function}_watts_syn_reg{lambda_l2}.csv")



    


0it [00:00, ?it/s]

128 300
Epoch 1/300
765/765 [==============================] - 148s 189ms/step - loss: 0.2684 - accuracy: 0.1268 - F1Score: 0.0261 - WeightedF1: 0.0432 - lr: 0.0010
Epoch 2/300
765/765 [==============================] - 146s 190ms/step - loss: 0.2540 - accuracy: 0.1404 - F1Score: 0.0453 - WeightedF1: 0.0716 - lr: 0.0010
Epoch 3/300
765/765 [==============================] - 145s 189ms/step - loss: 0.2471 - accuracy: 0.1455 - F1Score: 0.0617 - WeightedF1: 0.0949 - lr: 0.0010
Epoch 4/300
765/765 [==============================] - 144s 188ms/step - loss: 0.2423 - accuracy: 0.1452 - F1Score: 0.0782 - WeightedF1: 0.1174 - lr: 0.0010
Epoch 5/300
765/765 [==============================] - 144s 188ms/step - loss: 0.2383 - accuracy: 0.1443 - F1Score: 0.0917 - WeightedF1: 0.1356 - lr: 0.0010
Epoch 6/300
765/765 [==============================] - 145s 189ms/step - loss: 0.2348 - accuracy: 0.1480 - F1Score: 0.1027 - WeightedF1: 0.1497 - lr: 0.0010
Epoch 7/300
765/765 [=============================

### PC0

In [51]:

kf = KFold(n_splits=5, shuffle=True, random_state=RANDOM_SEED)
evaluation_results = []
predictions = []
class_weights_all = NUK.class_weights_tool(y)
debug = 0
for train_index, test_index in tqdm(kf.split(X)):
    print("Train indices:", train_index, "Test indices:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    X_train = normalize(X_train)
    X_test = normalize(X_test)

    # break

    model = NUK.PC0_reg(NmDevices, window_size, function, 128, k, lambda_l2=lambda_l2)
    # model = NUK.VGG11_1D(NmDevices, window_size)
    model.build((len(y_train) + len(y_test), window_size, 1))

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0003), loss='binary_crossentropy', metrics=[NUK.F1Score, NUK.WeightedF1Score(NUK.class_weights_tool(y_test))])
    lr_scheduler = LearningRateScheduler(scheduler)
    callback = tf.keras.callbacks.ReduceLROnPlateau(monitor='loss', factor=0.1,patience=2, min_lr=0.0000002)
    # for class weights
    # model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, class_weight=class_weights_UKD, callbacks=[lr_scheduler])
    # class_weights = NUK.class_weights_tool(y_train)
    model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, callbacks=[lr_scheduler])

    # Evaluate the model
    y_pred = model.predict(X_test)
    y_pred_tf = (y_pred > 0.5)

    predictions.append((y_pred_tf, y_test))

    # n_labels = y_test.shape[1]
    # co_occurrence_matrix = np.zeros((n_labels, n_labels))

    # for true, pred in zip(y_test, y_pred_tf):
    #     fn_labels = np.where((true == 1) & (pred == 0))[0]  # False negatives
    #     fp_labels = np.where((true == 0) & (pred == 1))[0]  # False positives

    #     for fn in fn_labels:
    #         for fp in fp_labels:
    #             co_occurrence_matrix[fn, fp] += 1



    report = metrics.classification_report(y_test, y_pred_tf, target_names=labels, zero_division=0, output_dict=True)
    print(report["weighted avg"])

    # report_DF = NUK.ClassificationReportToDF(report, labels)  
    # report_DF.rename(columns={report_DF.columns[0]: "device"}, inplace=True)
    evaluation_results.append(report)
    gc.collect()
    tf.keras.backend.clear_session()



    


0it [00:00, ?it/s]

Train indices: [    0     1     2 ... 49997 49998 49999] Test indices: [    4     6     7 ... 49988 49990 49992]
Epoch 1/20
313/313 [==============================] - 8s 17ms/step - loss: 0.3389 - F1Score: 4.3002e-05 - WeightedF1: 4.3906e-05 - lr: 3.0000e-04
Epoch 2/20
313/313 [==============================] - 5s 18ms/step - loss: 0.3194 - F1Score: 0.0114 - WeightedF1: 0.0118 - lr: 3.0000e-04
Epoch 3/20
313/313 [==============================] - 5s 17ms/step - loss: 0.3113 - F1Score: 0.0300 - WeightedF1: 0.0309 - lr: 3.0000e-04
Epoch 4/20
313/313 [==============================] - 6s 18ms/step - loss: 0.3057 - F1Score: 0.0460 - WeightedF1: 0.0472 - lr: 3.0000e-04
Epoch 5/20
313/313 [==============================] - 5s 18ms/step - loss: 0.2994 - F1Score: 0.0646 - WeightedF1: 0.0662 - lr: 3.0000e-04
Epoch 6/20
313/313 [==============================] - 5s 17ms/step - loss: 0.2919 - F1Score: 0.0894 - WeightedF1: 0.0915 - lr: 3.0000e-04
Epoch 7/20
313/313 [==============================]

KeyboardInterrupt: 

In [26]:

from tqdm.notebook import tqdm
import math
from matplotlib import pyplot as plt

data = X


count_zero = 0
count_nan = 0
count_neg = 0
count_inf = 0
n = range(len(X))
for i in tqdm(n):
    for j in range(len(data[i])):
        if math.isnan(data[i][j]) or np.isnan(data[i][j]):
            count_nan += 1
        elif data[i][j] < 0:
            count_neg += 1
        elif math.isinf(data[i][j]):
            count_inf += 1
        elif data[i][j] == 0:
            count_zero += 1

    
print("nan: ", count_nan, "neg: ", count_neg, "inf: ", count_inf, "zero: ", count_zero)  

  0%|          | 0/50000 [00:00<?, ?it/s]

nan:  0 neg:  0 inf:  0 zero:  824442


In [27]:
X.max(), X.min(), X.mean(), X.std()

(48642.98, 0.0, 484.20068986207036, 950.5494532852276)

In [16]:


# save with pickle
with open('predictions/predictions_real_normal.pkl', 'wb') as f:
    pickle.dump(predictions, f)



### Random forest

In [7]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
jobs = 110
kf = KFold(n_splits=5, shuffle=True, random_state=RANDOM_SEED)
evaluation_results = []

for train_index, test_index in tqdm(kf.split(X)):
    print("Train indices:", train_index, "Test indices:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    # Initialize the Random Forest classifier wrapped in a MultiOutputClassifier
    classifier = RandomForestClassifier(n_estimators=100, random_state=RANDOM_SEED, n_jobs=jobs)
    # classifier = MultiOutputClassifier(forest, n_jobs=-1)
    classifier.fit(X_train, y_train)

    # Predict the labels
    y_pred = classifier.predict(X_test)

    # Evaluate the model using classification report
    report = classification_report(y_test, y_pred, target_names=labels, zero_division=0, output_dict=True)

    evaluation_results.append(report)


for i in range(len(evaluation_results)):
    if i == 0:
        df = pd.DataFrame(evaluation_results[i]).T
    else:
        df = df + pd.DataFrame(evaluation_results[i]).T


df = df / 5
# df.to_csv(f"../../Energy_graph/data/results/w_size{window_size}_epochs{epochs}_batch_size{batch_size}_k{k}_classes{NmDevices}_staticLR_LSTM_watts_mixed100k.csv")
df.to_csv(f"../../Energy_graph/data/results/w_size{window_size}_classes{NmDevices}_watts_RF_mixed50k.csv")
    


0it [00:00, ?it/s]

Train indices: [     1      2      3 ... 147053 147054 147055] Test indices: [     0      4     12 ... 147049 147050 147052]


In [ ]:
evaluation_results = []
model = NUK.PC0(NmDevices, window_size, 'GRU',128, k)
model.build((len(y), window_size, 1))
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0003), loss='binary_crossentropy', metrics=[NUK.F1Score, NUK.WeightedF1Score(NUK.class_weights_tool(y_test))])
lr_scheduler = LearningRateScheduler(scheduler)

model.fit(X, y, batch_size=batch_size, epochs=epochs, callbacks=[lr_scheduler])


y_pred = model.predict(X_test)
y_pred_tf = (y_pred > 0.5)
report = metrics.classification_report(y_test, y_pred_tf, target_names=labels, zero_division=0, output_dict=True)

# report_DF = NUK.ClassificationReportToDF(report, labels)  
# report_DF.rename(columns={report_DF.columns[0]: "device"}, inplace=True)
df = pd.DataFrame(report)

### CatBoost

In [4]:
%pip install catboost
import catboost as cb


Note: you may need to restart the kernel to use updated packages.


In [15]:


kf = KFold(n_splits=5, shuffle=True, random_state=RANDOM_SEED)
evaluation_results = []
predictions = []
class_weights_all = NUK.class_weights_tool(y)
debug = 0
for train_index, test_index in tqdm(kf.split(X)):
    print("Train indices:", train_index, "Test indices:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    X_train = normalize(X_train)
    X_test = normalize(X_test)

    # break

    # Initialize CatBoost multilabel classifier
    model = cb.CatBoostClassifier(
        loss_function='MultiLogloss',
        verbose=500,
        random_seed=RANDOM_SEED,
        task_type="GPU",
        devices='0:1',
        # class_weights=class_weights_all,
        iterations=500,

        )  # Add other hyperparameters if needed

    # Train the model
    model.fit(X_train, y_train)

    # Get predictions
    y_pred = model.predict(X_test)

    # Convert predictions to boolean format similar to y_test for evaluation
    y_pred_tf = (y_pred == 1)
    print(y_pred)

    predictions.append((y_pred_tf, y_test))




    report = metrics.classification_report(y_test, y_pred_tf, target_names=labels, zero_division=0, output_dict=True)
    print(report["weighted avg"])
    break
    # report_DF = NUK.ClassificationReportToDF(report, labels)  
    # report_DF.rename(columns={report_DF.columns[0]: "device"}, inplace=True)
    evaluation_results.append(report)
    del model



    


0it [00:00, ?it/s]

Train indices: [    0     1     2 ... 49997 49998 49999] Test indices: [    4     6     7 ... 49988 49990 49992]
Learning rate set to 0.051213
0:	learn: 0.6478853	total: 545ms	remaining: 4m 32s
499:	learn: 0.2732736	total: 3m 57s	remaining: 0us
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
{'precision': 0.12109148155906953, 'recall': 0.021307596513075965, 'f1-score': 0.03282443454939719, 'support': 80300}


In [16]:
y_pred[2]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [57]:
gc.collect()
tf.keras.backend.clear_session()


### Evaluation

In [17]:
import os
for i in range(len(evaluation_results)):
    if i == 0:
        df = pd.DataFrame(evaluation_results[i]).T
    else:
        df = df + pd.DataFrame(evaluation_results[i]).T


df = df / 5
# check if directory exsits
if not os.path.exists(f"../../Energy_graph/data/results/{window_size}"):
    os.makedirs(f"../../Energy_graph/data/results/{window_size}")
if lambda_l2 == 0:
    df.to_csv(f"../../Energy_graph/data/results/{window_size}/Inception_epochs{epochs}_batch_size{batch_size}_k{k}_classes{NmDevices}_{function}_watts_real_norm.csv")
else:
    df.to_csv(f"../../Energy_graph/data/results/w_size{window_size}_epochs{epochs}_batch_size{batch_size}_k{k}_classes{NmDevices}_{function}_watts_syn_reg{lambda_l2}.csv")
# df.to_csv(f"../../Energy_graph/data/results/w_size{window_size}_classes{NmDevices}_watts_RF_mixed50k.csv")
    
# evaluation_results+ evaluation_results[1].to_